In [ ]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-nzs45y9k
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-nzs45y9k
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=6af67f070b99de1c4d98f965d0164fa9c2ab010321c6a3bbf5f3819d05f8119d
  Stored in directory: /tmp/pip-ephem-wheel-cache-xhs908i9/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <cuda.h>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#define m 10
__global__ void mul_r(int *a, int *b, int *c)
{
 	int tid = threadIdx.x;
 	if (tid < m)
  {
 		c[tid]= a[tid] * b[tid];
 	}
}
int main()
{
 	int  n, c, d, fst[10][10], snd[10][10], t_snd[10][10];
 	int row,col,sum_c, a[10], b[10], ans[10];
  n=m;
  for (c = 0; c < m; c++)
  {
    for (d = 0; d < n; d++)
 	  {
 		 fst[c][d]=rand()%10+1;
 		}
 	}
 	printf("display the elements of first matrix\n");	 
 	for (c = 0; c < m; c++) 
  {
    for (d = 0 ; d < n; d++) 
    {
 			printf("%d\t", fst[c][d]);
 		}
 		printf("\n");	 
 	}
  for (c = 0; c < m; c++)
  { 
    for (d = 0; d < n; d++)
 	  {
 		 snd[c][d]=rand()%10+1;
 		}
 	}
 	printf("display the elements of second matrix\n");	 
 	for (c = 0; c < m; c++) 
  {
    for (d = 0 ; d < n; d++) 
    {
 			printf("%d\t", snd[c][d]);
 		}
 		printf("\n");	 
 	}
 	for(c=0; c<m; c++)
  for(d=0; d<n; d++)
  {
    t_snd[d][c] = snd[c][d];
  }
  printf("\nTranspose of second Matrix:\n");
  for (c = 0; c < n; c++) 
  {
    for (d = 0 ; d < m; d++) 
    {
 			printf("%d\t", t_snd[c][d]);
 		}
 		printf("\n");	 
 	}
 	int *dev_a, *dev_b,*dev_ans;
 	cudaError_t err=cudaSuccess;
 	err=cudaMalloc((void**)&dev_a,m * sizeof(int)); 
 	if (err !=cudaSuccess)
 	{ 	
  printf("failed to  allocate on device \n");
 	printf("error is:\n",cudaGetErrorString(err));
 	exit(EXIT_FAILURE);
 	}
 	cudaMalloc((void**)&dev_b,m * sizeof(int)); 
 	cudaMalloc((void**)&dev_ans,m * sizeof(int)); 
 	row=0;
 	col=0;
  cudaEvent_t start, end;
  cudaEventCreate(&start);
  cudaEventCreate(&end);
  cudaEventRecord(start);	  
 	for(row=0; row<m; row++)
  {	
 	 for (d = 0 ; d < m; d++) 
   {
 		a[d]=fst[row][d];
 	 }
 	 cudaMemcpy(dev_a,a,m*sizeof(int), cudaMemcpyHostToDevice);	
 	 for (col=0; col<m; col++)
   {	
 	  for (d= 0 ; d < m; d++) 
    {
 			b[d]=t_snd[col][d];
 			ans[d]=0;
 		}
 		cudaMemcpy(dev_b,b,m*sizeof(int), cudaMemcpyHostToDevice);	
 		cudaMemcpy(dev_ans,ans,m*sizeof(int), cudaMemcpyHostToDevice);
 		mul_r<<<1,m>>>(dev_a,dev_b,dev_ans);
 		err=cudaMemcpy(ans,dev_ans,m*sizeof(int), cudaMemcpyDeviceToHost);
 		if (err !=cudaSuccess)
 		{ 	
        printf("failed to  copy from device \n");
 				exit(EXIT_FAILURE);
 		}
 	  sum_c=0;
 	  for (d = 0 ; d < m; d++) 
    {
 				sum_c+=ans[d];
 		}
 	  snd[row][col]=sum_c; 
 	 }
  }
 	cudaEventRecord(end);
  cudaEventSynchronize(end);
  float time = 0;
  cudaEventElapsedTime(&time, start, end);
  printf("execution time=%f\n",time);
 	printf(" Matrix multipliation ans=:\n");
  for (c = 0; c < n; c++) 
  {
    for (d = 0 ; d < m; d++) 
    {
 			printf("%d\t", snd[c][d]);
 		}
 		printf("\n");	 
 	}
 	return 0;
}

display the elements of first matrix
4	7	8	6	4	6	7	3	10	2	
3	8	1	10	4	7	1	7	3	7	
2	9	8	10	3	1	3	4	8	6	
10	3	3	9	10	8	4	7	2	3	
10	4	2	10	5	8	9	5	6	1	
4	7	2	1	7	4	3	1	7	2	
6	6	5	8	7	6	7	10	4	8	
5	6	3	6	5	8	5	5	4	1	
8	9	7	9	9	5	4	2	5	10	
3	1	7	9	10	3	7	7	5	10	
display the elements of second matrix
6	1	5	9	8	2	8	3	8	3	
3	7	2	1	7	2	6	10	5	10	
1	10	2	8	8	2	2	6	10	8	
8	7	8	4	7	6	7	4	10	5	
9	2	3	10	4	10	1	9	9	6	
1	10	7	4	9	6	7	2	2	6	
10	9	5	9	2	1	4	1	5	5	
5	5	8	7	4	2	8	6	10	7	
3	2	8	9	6	8	5	2	9	6	
10	8	6	4	9	9	4	2	9	10	

Transpose of second Matrix:
6	3	1	8	9	1	10	5	3	10	
1	7	10	7	2	10	9	5	2	8	
5	2	2	8	3	7	5	8	8	6	
9	1	8	4	10	4	9	7	9	4	
8	7	8	7	4	9	2	4	6	9	
2	2	2	6	10	6	1	2	8	9	
8	6	2	7	1	7	4	8	5	4	
3	10	6	4	9	2	1	6	2	2	
8	5	10	10	9	2	5	10	9	9	
3	10	8	5	6	6	5	7	6	10	
execution time=3.663232
 Matrix multipliation ans=:
278	357	303	377	361	261	288	251	428	372	
290	323	301	264	348	268	300	248	389	355	
289	342	287	316	358	263	274	268	451	385	
353	323	330	400	375	295	327	276	456	348	
350	344	352	406